In [1]:
import tensorflow as tf
from scipy import misc
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import facenet
import detect_face
import os
from os.path import join as pjoin
import sys
import time
import copy
import math
import pickle
from sklearn.svm import SVC
from sklearn.externals import joblib

/home/sohaib/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import pdb
from pathlib import Path

In [3]:
def show_img(im, figsize=None, ax=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    return ax

def draw_outline(o, lw):
    o.set_path_effects([patheffects.Stroke(linewidth=lw, foreground='black'), patheffects.Normal()])

def draw_rect(ax, b):
    patch = ax.add_patch(patches.Rectangle(b[:2], *b[-2:], fill=False, edgecolor='white', lw=2))
    
def draw_text(ax, xy, txt, sz=14):
    text = ax.text(*xy, txt, vericalalignment='top', color='white', fontsize=sz, weight='bold')
    draw_outline(text, 1)
    
def bb_hw(b):
    return np.array([b[0], b[1], b[2]-b[0]+1, b[3]-b[1]+1])

def crop(image,i):
    padding = 35
    b = bb_hw(bounding_boxes[i])

    x0 = int(b[0])-padding
    x0 = 0 if x0<0 else x0

    y0 = int(b[1])-padding
    y0 = 0 if y0<0 else y0

    width = int(b[2])+padding
    width = image.shape[0] if (width>image.shape[0]) else width

    height = int(b[3])+padding
    height = image.shape[1] if (height>image.shape[1]) else height

    return image[y0:y0+height , x0:x0+width, :]

In [4]:
PATH = Path('/media/alessa/Nouveau nom/data/parkar/tests/kaori_nicola')
# PATH = Path('/media/alessa/Nouveau nom/data/parkar/tests/alessa')
videoframes = list(PATH.iterdir())

output_dir = 'withpadd/kaori_nicola'

In [5]:
for frame_path in videoframes:
    frame = cv2.imread(str(frame_path))
    print(frame.shape)
    break

(720, 1080, 3)


In [6]:
print('Creating networks and loading parameters')
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    sess = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))
    with sess.as_default():
        threshold = [0.6, 0.7, 0.7]  # three steps's threshold
        minsize = 20  # minimum size of face
        factor = 0.709  # scale factor
        
        pnet, rnet, onet = detect_face.create_mtcnn(sess, '/home/sohaib/Desktop/Tools/real-time-deep-face-recognition/det_facenet')

        for frame_path in videoframes:
            frame = cv2.imread(str(frame_path))
            bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)

            nrof_faces = bounding_boxes.shape[0]
#             print('Detected_FaceNum: %d' % nrof_faces)        

#             ax = show_img(frame)
            nr_faces = bounding_boxes.shape[0]
            for i in range(nr_faces):
                b = bb_hw(bounding_boxes[i])
#                 draw_rect(ax, b)
                img = crop(frame, i)
                crop_name = str(frame_path).split('/')[-1].split('.')[0]
                cv2.imwrite(output_dir+'/'+str(crop_name)+'_'+str(i)+'.jpg', img)


Creating networks and loading parameters
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
